### Import libraries

In [12]:
import pandas as pd
import numpy as np

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

In [13]:
train = pd.read_csv("/Users/admin/_Work/Data/Practice/titanic/train.csv")
test = pd.read_csv("/Users/admin/_Work/Data/Practice/titanic/test.csv")

In [14]:
train.describe(include='all')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Braund, Mr. Owen Harris",male,NaN,NaN,NaN,347082,NaN,B96 B98,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [17]:
%%time
temp = train.groupby("Sex")['Age'].max().to_frame().reset_index()
temp

CPU times: user 4.39 ms, sys: 1.12 ms, total: 5.51 ms
Wall time: 6.78 ms


,Sex,Age
0,female,63.0
1,male,80.0


In [15]:
print(train.info())
print("*" * 40)
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
****************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Pass

In [86]:
def missing_percentage(df, show=True):
    total = df.isnull().sum().sort_values(ascending= False)
    percent = round(total / len(df)* 100, 2)

    return_df = pd.concat([total, percent], axis = 1, keys=['Total', 'Percent'])
    if show: print(return_df)

    return return_df

def percent_value_count(df, feature):
    total = df[feature].value_counts(dropna=False)
    percent = round(df[feature].value_counts(dropna= False, normalize=True) * 100, 2)

    return_df = pd.concat([total, percent], axis=1, keys=['Total', ' Percent'])

    return return_df

_ = missing_percentage(train)
_ = missing_percentage(test)


             Total  Percent
Cabin          687    77.10
Age            177    19.87
PassengerId      0     0.00
Survived         0     0.00
Pclass           0     0.00
Name             0     0.00
Sex              0     0.00
SibSp            0     0.00
Parch            0     0.00
Ticket           0     0.00
Fare             0     0.00
Embarked         0     0.00
             Total  Percent
Cabin          327    78.23
Age             86    20.57
Fare             1     0.24
PassengerId      0     0.00
Pclass           0     0.00
Name             0     0.00
Sex              0     0.00
SibSp            0     0.00
Parch            0     0.00
Ticket           0     0.00
Embarked         0     0.00


## Start filling empty value
Begin withh: 
- Embarked
- Age
- Cabin

### Working on Embarked

In [18]:
train['Embarked'].unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [20]:
train[train.Embarked.isna()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [67]:
fig = make_subplots(rows=1, cols=2, subplot_titles=["Training set", "Test set"])

df = [train, test]

for i in range(len(df)):
    for t in px.box(df[i].sort_values(by=['Embarked','Pclass']), x = 'Embarked', y = 'Fare', color = 'Pclass').data:
        fig.add_trace(t, row=(i//2) + 1, col=(i%2) + 1)

fig.update_layout(height=1000, width=800 ,boxmode='group').update_traces(showlegend=False, selector=lambda t: "Fare" not in t.hovertemplate)

fig.show()

In [77]:
train.iloc[np.where(train['Fare'] == train['Fare'].max())]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
258,259,1,1,"Ward, Miss. Anna",female,35.0,0,0,PC 17755,512.3292,NaN,C
679,680,1,1,"Cardeza, Mr. Thomas Drake Martinez",male,36.0,0,1,PC 17755,512.3292,B51 B53 B55,C
737,738,1,1,"Lesurer, Mr. Gustave J",male,35.0,0,0,PC 17755,512.3292,B101,C


In [74]:
train.iloc[np.where(train['Embarked'] == 'C')]['Fare'].mean()

59.95414404761905

## Fill missing Embraked value as C

In [76]:
train.Embarked.fillna('C', inplace=True)

## Working on Cabin

In [91]:
print(train.Cabin.isna().sum()/ len(train))
print(test.Cabin.isna().sum()/ len(test))

0.7710437710437711
0.7822966507177034


In [92]:
train.Cabin.sample(10)

269        C99
297    C22 C26
652        NaN
200        NaN
426        NaN
205         G6
60         NaN
316        NaN
590        NaN
810        NaN
Name: Cabin, dtype: object

## Merge train and test cabin

In [100]:
train_prefix = train.drop(columns=['Survived'])
all_data = pd.concat([train_prefix, test], axis=0)
all_data.fillna('N', inplace=True)
all_data.Cabin = [i[0] for i in all_data.Cabin]
all_data.sample(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
513,514,1,"Rothschild, Mrs. Martin (Elizabeth L. Barrett)",female,54.0,1,0,PC 17603,59.4,N,C
484,485,1,"Bishop, Mr. Dickinson H",male,25.0,1,0,11967,91.0792,B,C
300,1192,3,"Olsson, Mr. Oscar Wilhelm",male,32.0,0,0,347079,7.775,N,S
207,208,3,"Albimona, Mr. Nassef Cassem",male,26.0,0,0,2699,18.7875,N,C
183,184,2,"Becker, Master. Richard F",male,1.0,2,1,230136,39.0,F,S
87,979,3,"Badman, Miss. Emily Louisa",female,18.0,0,0,A/4 31416,8.05,N,S
182,183,3,"Asplund, Master. Clarence Gustaf Hugo",male,9.0,4,2,347077,31.3875,N,S
124,125,1,"White, Mr. Percival Wayland",male,54.0,0,1,35281,77.2875,D,S
488,489,3,"Somerton, Mr. Francis William",male,30.0,0,0,A.5. 18509,8.05,N,S
41,933,1,"Franklin, Mr. Thomas Parham",male,N,0,0,113778,26.55,D,S


In [101]:
percent_value_count(all_data, 'Cabin')

,Total,Percent
N,1014,77.46
C,94,7.18
B,65,4.97
D,46,3.51
E,41,3.13
A,22,1.68
F,21,1.60
G,5,0.38
T,1,0.08
